In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
bike = pd.read_csv('day.csv')

In [30]:
def preprocess_bike_data(bike):

    bike.drop(['instant'], axis=1, inplace=True) 
    bike.drop(['dteday'], axis=1, inplace=True)  
    bike.drop(['casual', 'registered'], axis=1, inplace=True) 
    
    bike = pd.get_dummies(bike, columns=['season', 'mnth', 'weekday', 'weathersit'], drop_first=True)
    
    scaler = StandardScaler()
    numerical_columns = ['temp', 'atemp', 'hum', 'windspeed']
    bike[numerical_columns] = scaler.fit_transform(bike[numerical_columns])
    
    return bike

processed_bike_data = preprocess_bike_data(bike)
processed_bike_data.head()


,yr,holiday,workingday,temp,atemp,hum,windspeed,cnt,season_2,season_3,...,mnth_11,mnth_12,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weathersit_2,weathersit_3
0,0,0,0,-0.826662,-0.679946,1.250171,-0.387892,985,False,False,...,False,False,False,False,False,False,False,True,True,False
1,0,0,0,-0.721095,-0.740652,0.479113,0.749602,801,False,False,...,False,False,False,False,False,False,False,False,True,False
2,0,0,1,-1.634657,-1.749767,-1.339274,0.746632,1349,False,False,...,False,False,True,False,False,False,False,False,False,False
3,0,0,1,-1.614780,-1.610270,-0.263182,-0.389829,1562,False,False,...,False,False,False,True,False,False,False,False,False,False
4,0,0,1,-1.467414,-1.504971,-1.341494,-0.046307,1600,False,False,...,False,False,False,False,True,False,False,False,False,False


In [31]:
bike.to_csv('preprocessed.csv', index=False)